In [1]:
import requests, re, datetime, pymysql
from bs4 import BeautifulSoup as BS
from pandas import Series, DataFrame
import pandas as pd

In [2]:
def open_url(url):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.103 Safari/537.36'}
    response = requests.get(url=url, headers=headers)
    response.encoding = 'utf-8'
    html = response.text
    return html

In [3]:
def get_cid(html):
    #用try防止有些av号没视频
    try:
        soup = BS(html, 'lxml')
        #视频名
        title = soup.select('div.v-title > h1')[0].get_text()
        #投稿人
        author = soup.select('meta[name="author"]')[0]['content']
        #弹幕的网站代码
        cid = re.findall(r'cid=(\d+)&', html)[0]
        print("{0}, {1}, cid:{2}".format(title, author, cid))
        return cid
    except:
        print('视频不见了哟')
        return False

In [4]:
def get_upload_date(html):
    #用try防止有些av号没视频
    try:
        soup = BS(html, 'lxml')
        #视频名
        title = soup.select('div.v-title > h1')[0].get_text()
        #投稿人
        author = soup.select('meta[name="author"]')[0]['content']
        #弹幕的网站代码
        upload_date = re.findall(r'datetime="(\d{4}-\d{2}-\d{2})', html)[0]
        print("{0}, {1}, upload_date:{2}".format(title, author, upload_date))
        return datetime.datetime.strptime(upload_date, "%Y-%m-%d")
    except:
        print('视频不见了哟')
        return False

In [5]:
def sec2str(seconds):
    seconds = eval(seconds)
    m, s = divmod(seconds, 60)
    h, m = divmod(m, 60)
    time = "%02d:%02d:%02d" % (h, m, s)
    return time

def stamp2date(timestamp):
    time = datetime.datetime.fromtimestamp(int(timestamp))
    return time.strftime('%Y-%m-%d %H:%M:%S')


In [6]:
def url_danmu_save(url_danmu, av_aid, av_cid):
    
    danmu_html = open_url(url=url_danmu)
    soup = BS(danmu_html, 'lxml')
    all_d = soup.select('d')
    
    config = {
          'host':'localhost',
          'port':3306,
          'user':'root',
          'password':'ms020312',
          'db':'test',
          'charset':'utf8mb4'
          }
    
    connection = pymysql.connect(**config)
    
    try:
        with connection.cursor() as cursor:
            N = 0
            now_str = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            for d in all_d:
                #把d标签中P的各个属性分离开
                danmu_list = d['p'].split(',')

                aid = av_aid
                cid = av_cid
                danmu_time = sec2str(danmu_list[0])
                danmu_type = danmu_list[1]
                danmu_size = danmu_list[2]
                danmu_color = danmu_list[3]
                danmu_stime = stamp2date(danmu_list[4])
                danmu_pole = danmu_list[5]
                sender_ID = danmu_list[6] 
                rowID = danmu_list[7] 
                danmu_message = d.get_text()
                record_time = now_str
                danmu_source = url_danmu
 
                
                
                #print(danmu_list[0])
                #print(danmu_message)

                sql = """INSERT INTO danmu(aid,cid,danmu_time,danmu_type,danmu_size,danmu_color,danmu_stime,danmu_pole,
                sender_ID,rowID,danmu_message,record_time,danmu_source)VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s);"""

                cursor.execute(sql,(
                aid,cid,danmu_time,danmu_type,danmu_size,danmu_color,danmu_stime,danmu_pole,
                sender_ID,rowID,danmu_message,record_time,danmu_source))

                N = N + 1

        connection.commit()
        print('INSET {0} ROWS in Table danmu'.format(N))
    finally:
        connection.close()

In [7]:
def CT_danmu():
    
    config = {
          'host':'localhost',
          'port':3306,
          'user':'root',
          'password':'ms020312',
          'db':'test',
          'charset':'utf8mb4'
          }
    
    connection = pymysql.connect(**config)

    try:
        with connection.cursor() as cursor:
            # Create a new record
            sql = """create table danmu(
            aid varchar(20), cid varchar(20),
            danmu_time varchar(20),
            danmu_type int, danmu_size int,
            danmu_color varchar(20),
            danmu_stime datetime,
            danmu_pole int,
            sender_ID varchar(20),
            rowID varchar(20),
            danmu_message varchar(200),
            record_time datetime,
            danmu_source varchar(60)
            )
            engine=innodb charset utf8mb4""" 
            cursor.execute(sql)

        connection.commit()
    finally:
        connection.close()

In [8]:
def danmu_save_last(av_aid):
    
    video_url ='http://www.bilibili.com/video/av{}/index_1.html'.format(av_aid)
    html = open_url(video_url)
    av_cid = get_cid(html)
    upload_date = get_upload_date(html)
    
    url_danmu = 'http://comment.bilibili.com/{}.xml'.format(av_cid)
    
    url_danmu_save(url_danmu, av_aid, av_cid)

In [9]:
def get_danmuhislist(av_cid):
    url_danmuhis = 'https://comment.bilibili.com/rolldate,{}'.format(av_cid)
    html = open_url(url_danmuhis)
    his_text = re.findall(r'{\S+?}',html)
    his_list = []
    for i in his_text:
        d = dict()
        d['timestamp'] = int(re.findall(r'timestamp":"(\d+)',i)[0])
        d['new'] = int(re.findall(r'new":"(\d+)',i)[0])
        his_list.append(d)
    his_list_DF = DataFrame(his_list)
    return his_list_DF

In [10]:
def danmu_save_all(av_aid, av_cid):
        
    url_danmu = 'http://comment.bilibili.com/{}.xml'.format(av_cid)
    print('saving url: %s' % (url_danmu))
    url_danmu_save(url_danmu, av_aid, av_cid)
    print('current danmu_his been saved')
    
    
    danmu_hislist = get_danmuhislist(av_cid)
    N = 0
    for x in danmu_hislist.eewwweeeeeeee:
        url_danmu = 'https://comment.bilibili.com/dmroll,%s,%s' % (x, av_cid)
        print('saving url: %s' % (url_danmu))
        url_danmu_save(url_danmu, av_aid, av_cid)
        N = N + 1
    print('%d danmu_his been saved' % (N))
    
    print('aid:%s cid:%s has been saved' % (av_aid, av_cid))

In [11]:
def danmu_save_allpage(av_aid):
    
    video_url ='http://www.bilibili.com/video/av{}/index_1.html'.format(av_aid)
    html = open_url(video_url)
    try:   
        soup = BS(html, 'lxml')
        title = soup.select('div.v-title > h1')[0].get_text()
        #投稿人
        author = soup.select('meta[name="author"]')[0]['content']
        #弹幕的网站代码
        upload_date = re.findall(r'datetime="(\d{4}-\d{2}-\d{2})', html)[0]
        #上传视频时间
        print("{0}, {1}, upload_date:{2}".format(title, author, upload_date))
    except:
        print('视频不见了哟')
        return False
        
    
    p_list = soup.select('div.v-plist > div > select > option')
    if len(p_list) == 0:
        av_cid = re.findall(r'cid=(\d+)&', html)[0]
        print('单P视频，cid：%s' % (av_cid))
        danmu_save_all(av_aid, av_cid)
    else:
        av_cid_list = []
        for i in p_list:
            av_cid = i.get_attribute_list('cid')[0]
            av_cid_list.append(av_cid)
        print('%s P视频，cid_list：%s' % (len(p_list),av_cid_list))
        for av_cid in av_cid_list:
            danmu_save_all(av_aid, av_cid)


In [14]:
av_aid = 20843765

In [24]:
av_aid_list = [17773532, 17751783]

In [25]:
for aid in av_aid_list:
    danmu_save_allpage(aid)

【STN快报】屎蛋汗虚沧的游戏推荐（必看, STN工作室, upload_date:2017-12-30
单P视频，cid：29018504
saving url: http://comment.bilibili.com/29018504.xml
INSET 1000 ROWS in Table danmu
current danmu_his been saved
saving url: https://comment.bilibili.com/dmroll,1514563200,29018504
INSET 1000 ROWS in Table danmu
saving url: https://comment.bilibili.com/dmroll,1514649600,29018504
INSET 1000 ROWS in Table danmu
saving url: https://comment.bilibili.com/dmroll,1514736000,29018504
INSET 1000 ROWS in Table danmu
saving url: https://comment.bilibili.com/dmroll,1514822400,29018504
INSET 1000 ROWS in Table danmu
saving url: https://comment.bilibili.com/dmroll,1514908800,29018504
INSET 1000 ROWS in Table danmu
saving url: https://comment.bilibili.com/dmroll,1514995200,29018504
INSET 1000 ROWS in Table danmu
saving url: https://comment.bilibili.com/dmroll,1515081600,29018504
INSET 1000 ROWS in Table danmu
saving url: https://comment.bilibili.com/dmroll,1515168000,29018504
INSET 1000 ROWS in Table danmu
saving url: https://comm

INSET 3000 ROWS in Table danmu
saving url: https://comment.bilibili.com/dmroll,1515254400,28981200
INSET 3000 ROWS in Table danmu
saving url: https://comment.bilibili.com/dmroll,1515340800,28981200
INSET 3000 ROWS in Table danmu
saving url: https://comment.bilibili.com/dmroll,1515427200,28981200
INSET 3000 ROWS in Table danmu
saving url: https://comment.bilibili.com/dmroll,1515513600,28981200
INSET 3000 ROWS in Table danmu
saving url: https://comment.bilibili.com/dmroll,1515600000,28981200
INSET 3000 ROWS in Table danmu
saving url: https://comment.bilibili.com/dmroll,1515686400,28981200
INSET 3000 ROWS in Table danmu
saving url: https://comment.bilibili.com/dmroll,1515772800,28981200
INSET 3000 ROWS in Table danmu
saving url: https://comment.bilibili.com/dmroll,1515859200,28981200
INSET 3000 ROWS in Table danmu
saving url: https://comment.bilibili.com/dmroll,1515945600,28981200
INSET 3000 ROWS in Table danmu
saving url: https://comment.bilibili.com/dmroll,1516032000,28981200
INSET 3000